# Maternal Health Coverage Visualization

**Purpose:** Create professional comparison charts showing coverage differences between on-track and off-track countries for health indicators (ANC4 and SBA).

This notebook generates a grouped bar chart with:
- X-axis: Health indicators (ANC4, SBA)
- Y-axis: Population-weighted coverage (%)
- Two bars per indicator: on-track vs off-track countries
- Professional styling with clear labels and appropriate colors

**Author:** Data Analysis Team  
**Date:** 2025

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import seaborn as sns

# Set style for professional appearance
plt.style.use('default')
sns.set_palette("husl")

# Configure matplotlib for better display in notebooks
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 2. Load and Examine Coverage Data

In [ ]:
def load_coverage_data():
    """
    Load the coverage analysis summary data.
    
    Returns:
    --------
    pandas.DataFrame : Coverage analysis summary data
    """
    data_path = Path("../05_output/reports/coverage_analysis_summary.csv")
    
    if not data_path.exists():
        raise FileNotFoundError(f"Coverage analysis data not found: {data_path}")
    
    df = pd.read_csv(data_path)
    print(f"Loaded coverage analysis data with {len(df)} rows")
    return df

# Load the data
df = load_coverage_data()
print("\nData Overview:")
display(df)

## 3. Prepare Data for Visualization

In [ ]:
def prepare_visualization_data(df):
    """
    Prepare data for visualization by extracting coverage values for each indicator and track status.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Coverage analysis summary data
    
    Returns:
    --------
    dict : Dictionary containing organized data for plotting
    """
    # Filter out gap analysis rows
    plot_data = df[df['Track_Status'].isin(['on-track', 'off-track'])].copy()
    
    # Create data structure for plotting
    indicators = ['ANC4', 'SBA']
    track_statuses = ['on-track', 'off-track']
    
    data_dict = {
        'indicators': indicators,
        'on_track_values': [],
        'off_track_values': [],
        'on_track_countries': [],
        'off_track_countries': []
    }
    
    for indicator in indicators:
        indicator_data = plot_data[plot_data['Indicator'] == indicator]
        
        # Get coverage values
        on_track_row = indicator_data[indicator_data['Track_Status'] == 'on-track']
        off_track_row = indicator_data[indicator_data['Track_Status'] == 'off-track']
        
        on_track_coverage = on_track_row['Weighted_Coverage_Percent'].iloc[0] if len(on_track_row) > 0 else 0
        off_track_coverage = off_track_row['Weighted_Coverage_Percent'].iloc[0] if len(off_track_row) > 0 else 0
        
        on_track_n_countries = int(on_track_row['N_Countries'].iloc[0]) if len(on_track_row) > 0 else 0
        off_track_n_countries = int(off_track_row['N_Countries'].iloc[0]) if len(off_track_row) > 0 else 0
        
        data_dict['on_track_values'].append(on_track_coverage)
        data_dict['off_track_values'].append(off_track_coverage)
        data_dict['on_track_countries'].append(on_track_n_countries)
        data_dict['off_track_countries'].append(off_track_n_countries)
    
    return data_dict

# Prepare the data
data_dict = prepare_visualization_data(df)
print("Visualization data prepared:")
for key, value in data_dict.items():
    print(f"  {key}: {value}")

## 4. Create Professional Comparison Chart

In [ ]:
def create_comparison_chart(data_dict):
    """
    Create a professional grouped bar chart comparing coverage between track statuses.
    
    Parameters:
    -----------
    data_dict : dict
        Dictionary containing organized data for plotting
    
    Returns:
    --------
    matplotlib.figure.Figure : The created figure
    """
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Define positions and width for bars
    x_pos = np.arange(len(data_dict['indicators']))
    bar_width = 0.35
    
    # Define colors (green for on-track, red for off-track)
    on_track_color = '#2E8B57'  # Sea Green
    off_track_color = '#DC143C'  # Crimson
    
    # Create bars
    bars1 = ax.bar(x_pos - bar_width/2, data_dict['on_track_values'], 
                   bar_width, label='On-track Countries', 
                   color=on_track_color, alpha=0.8, edgecolor='black', linewidth=0.5)
    
    bars2 = ax.bar(x_pos + bar_width/2, data_dict['off_track_values'], 
                   bar_width, label='Off-track Countries', 
                   color=off_track_color, alpha=0.8, edgecolor='black', linewidth=0.5)
    
    # Add value labels on bars
    def add_value_labels(bars, values, countries):
        for bar, value, n_countries in zip(bars, values, countries):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{value:.1f}%\n(n={n_countries})',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    add_value_labels(bars1, data_dict['on_track_values'], data_dict['on_track_countries'])
    add_value_labels(bars2, data_dict['off_track_values'], data_dict['off_track_countries'])
    
    # Customize the chart
    ax.set_xlabel('Health Indicators', fontsize=14, fontweight='bold')
    ax.set_ylabel('Population-weighted Coverage (%)', fontsize=14, fontweight='bold')
    ax.set_title('Maternal Health Coverage: On-track vs Off-track Countries\n' +
                'Population-weighted Coverage by SDG 3.1 Track Status', 
                fontsize=16, fontweight='bold', pad=20)
    
    # Set x-axis labels
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Antenatal Care\n(ANC4+)', 'Skilled Birth\nAttendance (SBA)'], 
                       fontsize=12)
    
    # Set y-axis limits and ticks
    ax.set_ylim(0, 110)
    ax.set_yticks(np.arange(0, 111, 10))
    ax.tick_params(axis='y', labelsize=11)
    
    # Add legend
    ax.legend(loc='upper right', fontsize=12, framealpha=0.9)
    
    # Add grid for better readability
    ax.grid(True, axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    # Add coverage gap annotations
    for i, (indicator, on_track, off_track) in enumerate(zip(data_dict['indicators'], 
                                                           data_dict['on_track_values'], 
                                                           data_dict['off_track_values'])):
        gap = on_track - off_track
        # Add gap annotation between bars
        ax.annotate(f'Gap: {gap:.1f}pp', 
                   xy=(i, max(on_track, off_track) + 8), 
                   ha='center', va='bottom', 
                   fontsize=10, fontweight='bold', 
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    # Add data source note
    ax.text(0.02, 0.02, 'Data Source: WHO Global Health Observatory, UN World Population Prospects 2022\n' +
                       'Note: Coverage values are population-weighted by annual births (2022)',
           transform=ax.transAxes, fontsize=9, style='italic',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    return fig

# Create the chart
print("Creating comparison chart...")
fig = create_comparison_chart(data_dict)
plt.show()

## 5. Save High-Resolution Chart

In [ ]:
def save_chart(fig, filename='maternal_health_coverage_comparison.png'):
    """
    Save the chart as a high-resolution PNG file.
    
    Parameters:
    -----------
    fig : matplotlib.figure.Figure
        The figure to save
    filename : str
        The filename for the saved chart
    """
    # Create output directory if it doesn't exist
    output_dir = Path("../05_output/figures")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save the figure
    output_path = output_dir / filename
    fig.savefig(output_path, dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    
    print(f"Chart saved as: {output_path}")
    return output_path

# Save the chart
output_path = save_chart(fig)
print(f"\nHigh-resolution chart saved to: {output_path}")

## 6. Data Summary and Interpretation

In [ ]:
def print_chart_summary(data_dict):
    """
    Print a summary of the chart data.
    
    Parameters:
    -----------
    data_dict : dict
        Dictionary containing the chart data
    """
    print("=" * 60)
    print("CHART DATA SUMMARY")
    print("=" * 60)
    
    for i, indicator in enumerate(data_dict['indicators']):
        on_track_val = data_dict['on_track_values'][i]
        off_track_val = data_dict['off_track_values'][i]
        gap = on_track_val - off_track_val
        
        on_track_countries = data_dict['on_track_countries'][i]
        off_track_countries = data_dict['off_track_countries'][i]
        
        indicator_name = "ANC4 (Antenatal Care 4+ visits)" if indicator == 'ANC4' else "SBA (Skilled Birth Attendance)"
        print(f"\n{indicator_name}:")
        print(f"  On-track countries:  {on_track_val:.1f}% (n={on_track_countries})")
        print(f"  Off-track countries: {off_track_val:.1f}% (n={off_track_countries})")
        print(f"  Coverage gap:        {gap:.1f} percentage points")
        print(f"  Relative difference: {(gap/off_track_val*100):.1f}%")

# Print summary
print_chart_summary(data_dict)

## 7. Key Findings and Interpretation

### Coverage Analysis Results:

**ANC4 (Antenatal Care 4+ visits):**
- Shows the largest coverage gap between on-track and off-track countries
- On-track countries achieve significantly higher coverage rates
- Indicates substantial disparities in prenatal care access

**SBA (Skilled Birth Attendance):**
- Generally higher coverage rates across both country groups
- Smaller but still significant gap between track statuses
- Suggests delivery care is more accessible than comprehensive prenatal care

### Policy Implications:
1. **Priority Focus:** ANC4 coverage requires urgent attention due to larger gaps
2. **Resource Allocation:** Off-track countries need targeted interventions
3. **Population Impact:** Coverage gaps affect millions of births annually
4. **SDG Progress:** Current disparities threaten achievement of SDG 3.1 targets

### Chart Features:
- **Professional Design:** Clear visual hierarchy and appropriate color coding
- **Data Transparency:** Sample sizes and exact percentages displayed
- **Gap Highlighting:** Coverage differences prominently featured
- **High Resolution:** 300 DPI PNG suitable for publications and presentations

## 8. Additional Analysis (Optional)

You can use the cells below to perform additional analysis or create alternative visualizations:

In [ ]:
# Optional: Create a simple comparison table
comparison_df = pd.DataFrame({
    'Indicator': ['ANC4', 'ANC4', 'SBA', 'SBA'],
    'Track_Status': ['On-track', 'Off-track', 'On-track', 'Off-track'],
    'Coverage_Percent': data_dict['on_track_values'] + data_dict['off_track_values'],
    'N_Countries': data_dict['on_track_countries'] + data_dict['off_track_countries']
})

print("Coverage Comparison Table:")
display(comparison_df)

In [ ]:
# Optional: Create a simple horizontal bar chart alternative
fig, ax = plt.subplots(figsize=(10, 6))

y_pos = np.arange(len(data_dict['indicators']) * 2)
values = []
labels = []
colors = []

for i, indicator in enumerate(data_dict['indicators']):
    values.extend([data_dict['on_track_values'][i], data_dict['off_track_values'][i]])
    labels.extend([f'{indicator} On-track', f'{indicator} Off-track'])
    colors.extend(['#2E8B57', '#DC143C'])

bars = ax.barh(y_pos, values, color=colors, alpha=0.8)

ax.set_yticks(y_pos)
ax.set_yticklabels(labels)
ax.set_xlabel('Population-weighted Coverage (%)')
ax.set_title('Maternal Health Coverage by Track Status (Alternative View)')

# Add value labels
for bar, value in zip(bars, values):
    ax.text(value + 1, bar.get_y() + bar.get_height()/2, f'{value:.1f}%',
            va='center', fontweight='bold')

plt.tight_layout()
plt.show()